### Access Token Generation.


#### Use your user access token to connect to the server. This will mask your user credentials.There are different ways to generate the token. The token expires after 12 hours. The following examples shows how to retrieve a bearer token from the user management service.



### User Access Token

In [2]:
import os
mltoken =  os.environ['USER_ACCESS_TOKEN']

### Define the Credentials

In [1]:
import getpass 
from getpass import getpass

user = input('user: ')
pwd = getpass(f'password: ')

user:  nijesh
password:  ········


### Curl Command

In [4]:
!curl -k -X GET https://dhl-cpd-dhl.apps.anup-dal12-demo-cpdga.demo.ibmcloud.com/v1/preauth/validateAuth -u "$user":"$pwd"


{"username":"nijesh","role":"Admin","permissions":["administrator","can_provision","manage_catalog","manage_information_assets","manage_quality","manage_discovery","manage_metadata_import","manage_categories","author_governance_artifacts","manage_governance_workflow","access_catalog","access_information_assets","view_quality","access_quality"],"sub":"nijesh","iss":"KNOXSSO","aud":"DSX","uid":"1000331001","authenticator":"default","accessToken":"eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VybmFtZSI6Im5pamVzaCIsInJvbGUiOiJBZG1pbiIsInBlcm1pc3Npb25zIjpbImFkbWluaXN0cmF0b3IiLCJjYW5fcHJvdmlzaW9uIiwibWFuYWdlX2NhdGFsb2ciLCJtYW5hZ2VfaW5mb3JtYXRpb25fYXNzZXRzIiwibWFuYWdlX3F1YWxpdHkiLCJtYW5hZ2VfZGlzY292ZXJ5IiwibWFuYWdlX21ldGFkYXRhX2ltcG9ydCIsIm1hbmFnZV9jYXRlZ29yaWVzIiwiYXV0aG9yX2dvdmVybmFuY2VfYXJ0aWZhY3RzIiwibWFuYWdlX2dvdmVybmFuY2Vfd29ya2Zsb3ciLCJhY2Nlc3NfY2F0YWxvZyIsImFjY2Vzc19pbmZvcm1hdGlvbl9hc3NldHMiLCJ2aWV3X3F1YWxpdHkiLCJhY2Nlc3NfcXVhbGl0eSJdLCJzdWIiOiJuaWplc2giLCJpc3MiOiJLTk9YU1NPIiwiYXVkIjoiRF

### The Elegant Python way

In [2]:
CONNECTION_CREDENTIALS = {
                      "username": user ,
                      "password": pwd,
                      # address should be replaced with ip, port pair to be used in scripts outside ICP
                      "url": "https://dhl-cpd-dhl.apps.anup-dal12-demo-cpdga.demo.ibmcloud.com"
                   }

In [3]:
import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time
import warnings
warnings.filterwarnings("ignore")

def generate_access_token():
    headers={}
    headers["Accept"] = "application/json"
    auth = HTTPBasicAuth(CONNECTION_CREDENTIALS["username"], CONNECTION_CREDENTIALS["password"])
    ICP_TOKEN_URL= CONNECTION_CREDENTIALS["url"] + "/v1/preauth/validateAuth"
    response = requests.get(ICP_TOKEN_URL, headers=headers, auth=auth, verify=False)
    json_data = response.json()
    icp_access_token = json_data['accessToken']
    return icp_access_token


In [8]:
mltoken = generate_access_token()
mltoken

### Demonstrate the Scoring of Endpoint 

### Imports

In [9]:
import os
import pandas as pd
from cpd_utils import *

### Prepare Scoring Payload

In [10]:
payload= prepare_scoring_payload(path="/project_data/data_asset/trn_dataset_transformed.tsv",no_of_records=2)
    
#payload = prepare_scoring_payload("/project_data/data_asset/eval_mdd.tsv",no_of_records=1)
values = payload['input_data'][0]['values']
fields = payload['input_data'][0]['fields']


### Use the Requests Library and Authenticated mltoken to fetch the score using the model endpoint

In [12]:
import urllib3, requests, json
import warnings
warnings.filterwarnings("ignore")

# NOTE: you must construct mltoken based on provided documentation	
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"input_data": [{"fields":fields, "values":values}]}

#Model V1

response_scoring = requests.post('https://dhl-cpd-dhl.apps.anup-dal12-demo-cpdga.demo.ibmcloud.com/v4/deployments/6058c31f-f7f8-4047-b3d5-457ddca47d21/predictions', json=payload_scoring, headers=header,verify=False)
print("Scoring response")
print(json.loads(response_scoring.text))

Scoring response
{'predictions': [{'fields': ['offer_id', 'organization_id', 'discount', 'recommended_revenue', 'recommended_pid', 'recommended_pid_name', 'physical_channel', 'published_revenue', 'new_potential_revenue', 'shipments', 'wps', 'product_cluster', 'industry_code', 'contactrole', 'reason_for_lead', 'lead_source_type', 'primary_competitor', 'predictions'], 'values': [['CL_O_114028920439', 'DHL CL', 54.99967567667746, 69.18017647058825, nan, 'TDI_45%', 'E-Commerce', 129.5004, 66.54, 2, 30.0, 'TDImport', 'FASHFASH', 'Influencer', 'Campaign', 'Liveball', 'FedEx', 51.72905235791909], ['CL_O_114028025699', 'DHL CL', 50.0, 58.436470588235295, nan, 'TDE_42%', 'Freight Forwarders', 103.1832, 61.2, 2, 30.0, 'TDExport', 'CSMRCSDU', 'Influencer', 'Cash Customer', 'CSV', 'UPS', 47.81647307705444]]}]}


### Model V2

In [13]:
import urllib3, requests, json
# NOTE: you must construct mltoken based on provided documentation	
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

response_scoring = requests.post('https://dhl-cpd-dhl.apps.anup-dal12-demo-cpdga.demo.ibmcloud.com/v4/deployments/2aa82e1a-20a5-43cd-b5df-16a634a93237/predictions', json=payload_scoring, headers=header,verify=False)
print("Scoring response")
print(json.loads(response_scoring.text))

Scoring response
{'predictions': [{'fields': ['offer_id', 'organization_id', 'discount', 'recommended_revenue', 'recommended_pid', 'recommended_pid_name', 'physical_channel', 'published_revenue', 'new_potential_revenue', 'shipments', 'wps', 'product_cluster', 'industry_code', 'contactrole', 'reason_for_lead', 'lead_source_type', 'primary_competitor', 'predictions'], 'values': [['CL_O_114028920439', 'DHL CL', 54.99967567667746, 69.18017647058825, nan, 'TDI_45%', 'E-Commerce', 129.5004, 66.54, 2, 30.0, 'TDImport', 'FASHFASH', 'Influencer', 'Campaign', 'Liveball', 'FedEx', 54.4371805832009], ['CL_O_114028025699', 'DHL CL', 50.0, 58.436470588235295, nan, 'TDE_42%', 'Freight Forwarders', 103.1832, 61.2, 2, 30.0, 'TDExport', 'CSMRCSDU', 'Influencer', 'Cash Customer', 'CSV', 'UPS', 47.31853549082904]]}]}
